In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
city_data = pd.read_csv("../Output_Data/Cities.csv")
city_data = city_data.drop("Unnamed: 0", axis=1)
city_data.head(10)

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,London,51.51,-0.13,45.00,81,91,9.17,GB,1607572217
1,New York,40.71,-74.01,37.99,74,90,9.17,US,1607572149
2,Miami,25.77,-80.19,51.80,62,1,7.85,US,1607572512
3,San Diego,32.72,-117.16,60.80,82,75,0.98,US,1607572694
4,Sydney,-33.87,151.21,73.99,77,40,33.33,AU,1607572632
5,Brasília,-15.78,-47.93,68.00,88,40,4.70,BR,1607572708


In [4]:
cities = city_data[["Latitude", "Longitude"]]
humidity = city_data["Humidity"].astype(float)

In [19]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(cities, weights = humidity, dissipating=False, max_intensity =100, point_radius=5)
figure_layout = {'width': '100px', 'margin': '0 auto 0 auto'}
gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
city_data[(city_data["Max Temp"] >= 70) & (city_data["Max Temp"] < 80) & 
                          (city_data["Wind Speed"] < 10) & (city_data["Cloudiness"] == 0)]

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


In [7]:
# ideal_weather = city_data[(city_data["Max Temp"] >= 70) & (city_data["Max Temp"] < 80) & 
#                           (city_data["Wind Speed"] < 10) & (city_data["Cloudiness"] == 0)]
# ideal_weather
ideal_weather = city_data[(city_data["Max Temp"] >= 0) & (city_data["Max Temp"] < 100) & 
                          (city_data["Wind Speed"] > 0) & (city_data["Cloudiness"] != 0)]
ideal_weather.head(10)

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,London,51.51,-0.13,45.00,81,91,9.17,GB,1607572217
1,New York,40.71,-74.01,37.99,74,90,9.17,US,1607572149
2,Miami,25.77,-80.19,51.80,62,1,7.85,US,1607572512
3,San Diego,32.72,-117.16,60.80,82,75,0.98,US,1607572694
4,Sydney,-33.87,151.21,73.99,77,40,33.33,AU,1607572632
5,Brasília,-15.78,-47.93,68.00,88,40,4.70,BR,1607572708


In [11]:
#adding column to data set
ideal_weather["Hotel Name"] = ""

In [14]:
radius = "5000"
place_type = "lodging"

for index, city in ideal_weather.iterrows():
    latitude = city["Latitude"]
    longitude = city["Longitude"]
    location = f"{latitude},{longitude}"
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type={place_type}&key={g_key}"
    response = requests.get(url).json()
    try:
        #print(url)
        hotel_name = response["results"][0]["name"]
        ideal_weather.loc[index, "Hotel Name"] = hotel_name
        
    except KeyError:
        continue
ideal_weather

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,London,51.51,-0.13,45.00,81,91,9.17,GB,1607572217,"Radisson Blu Edwardian, Hampshire"
1,New York,40.71,-74.01,37.99,74,90,9.17,US,1607572149,Millennium Hilton New York Downtown
2,Miami,25.77,-80.19,51.80,62,1,7.85,US,1607572512,Kimpton EPIC Hotel
3,San Diego,32.72,-117.16,60.80,82,75,0.98,US,1607572694,Hilton San Diego Bayfront
4,Sydney,-33.87,151.21,73.99,77,40,33.33,AU,1607572632,QT Sydney
5,Brasília,-15.78,-47.93,68.00,88,40,4.70,BR,1607572708,Grand Mercure Brasilia Eixo Monumental


In [49]:
hotel_df = ideal_weather[["Hotel Name","City","Country","Latitude","Longitude"]]
hotel_df = hotel_df.rename(columns = {"Latitude":"Lat", "Longitude":"Lng"})
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,"Radisson Blu Edwardian, Hampshire",London,GB,51.51,-0.13
1,Millennium Hilton New York Downtown,New York,US,40.71,-74.01
2,Kimpton EPIC Hotel,Miami,US,25.77,-80.19
3,Hilton San Diego Bayfront,San Diego,US,32.72,-117.16
4,QT Sydney,Sydney,AU,-33.87,151.21
5,Grand Mercure Brasilia Eixo Monumental,Brasília,BR,-15.78,-47.93


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [56]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(cities, weights = humidity, dissipating=False, max_intensity =10, point_radius=1)
fig.add_layer(heat_layer)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))